<h1>Processamento e Modelação de Big Data</h1>

In [ ]:
#!pip install ipynb

<h3> Importes </h3>

In [ ]:
from ipynb.fs.full.DataAnalysis import DataAnalysis
from ipynb.fs.full.MLModel import MLModel
import seaborn as sns
import findspark
findspark.init()
import pyspark
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession\
    .builder\
    .appName("BigData")\
    .config("spark.sql.shuffle.partitions",6)\
    .getOrCreate()

<h3> Leitura do ficheiro CSV com o Spark </h3>

In [ ]:
df = spark.read.format("csv").option("header","true").load("train.csv")

<h3> Criação do objecto Data Analysis e teste ao núcleo do Spark </h3>

In [ ]:
da = DataAnalysis(df,spark)
da.getSpark()

<h1> Análise Exploratória </h1>

<h3> Total do número de registos inicial do Dataset </h3>

In [ ]:
da.getRecordsCount()

<h3> Total do número de colunas inicial do Dataset </h3>

In [ ]:
len(da.getCols())

<h3> Eliminação das colunas que contêm mais de 25% de valores nulos</h3>

In [ ]:
da.dropColNA(0.25)

<h3> Total do número de colunas após eliminação </h3>

In [ ]:
len(da.getCols())

<h3> Eliminação da coluna MachineIdentifier </h3>

In [ ]:
da.dropCol('MachineIdentifier')

In [ ]:
len(da.getCols())

<h3> Eliminação das linhas que contêm pelo menos uma das colunas com valor nulo </h3>

In [ ]:
da.dropRowNA()

<h3>Contagem do número de registos após eliminação </h3>

In [ ]:
da.getRecordsCount()

<h3>Análise gráfica acerca da distribuição de valores de uma dada coluna(Exemplo: Firewall)</h3>

In [ ]:
da.getCountPlot("Firewall")

<h3>Eliminação dos registos que contêm o valor de -1 das colunas que tratam acerca da resolução do ecrã</h3>

In [ ]:
da.dropValFromCol("Census_InternalPrimaryDisplayResolutionVertical",-1)
da.dropValFromCol("Census_InternalPrimaryDisplayResolutionHorizontal",-1)

In [ ]:
da.getRecordsCount()

<h3>Transformação dos dados, String to Int</h3>

In [ ]:
col_types = {"IsBeta":"int", "RtpStateBitfield":"int", "IsSxsPassiveMode":"int", "AVProductStatesIdentifier":"int",\
                     "AVProductsInstalled":"int" , "AVProductsEnabled":"int","HasTpm":"int","CountryIdentifier":"int",\
                     "CityIdentifier":"int","GeoNameIdentifier":"int","LocaleEnglishNameIdentifier":"int",\
                     "OsBuild":"int","OsSuite":"int","IsProtected":"int",\
                     "AutoSampleOptIn":"int", "SMode":"int", "IeVerIdentifier":"int", "Firewall":"int", 
                     "UacLuaenable":"int","Census_OEMNameIdentifier":"int","Census_OEMModelIdentifier":"int",\
                     "Census_ProcessorCoreCount":"int","Census_ProcessorManufacturerIdentifier":"int",\
                     "Census_ProcessorModelIdentifier":"int","Census_PrimaryDiskTotalCapacity":"double",\
                     "Census_SystemVolumeTotalCapacity":"int","Census_HasOpticalDiskDrive":"int",\
                     "Census_TotalPhysicalRAM":"int", "Census_InternalPrimaryDiagonalDisplaySizeInInches":"double", \
                     "Census_InternalPrimaryDisplayResolutionHorizontal":"int", \
                     "Census_InternalPrimaryDisplayResolutionVertical": "int", "Census_InternalBatteryNumberOfCharges":"double",\
                     "Census_OSBuildNumber":"int", "Census_OSBuildRevision":"int", "Census_OSInstallLanguageIdentifier":"int",\
                     "Census_OSUILocaleIdentifier":"int", "Census_IsPortableOperatingSystem": "int",\
                     "Census_IsFlightsDisabled":"int","Census_FirmwareManufacturerIdentifier":"int",\
                     "Census_FirmwareVersionIdentifier":"int","Census_IsSecureBootEnabled":"int",\
                     "Census_IsVirtualDevice":"int","Census_IsTouchEnabled":"int","Census_IsPenCapable":"int",\
                     "Census_IsAlwaysOnAlwaysConnectedCapable":"int","Wdft_IsGamer":"int","Wdft_RegionIdentifier":"int",\
                     "HasDetections":"int"}



da.changeColDataTypes(col_types)


<h3>Transformação dos dados, Categorical to Int (Label Enconding / Ordinal Enconding)</h3>

In [ ]:
string_to_int = {"ProductName":"int","EngineVersion":"int","AppVersion":"int", "AvSigVersion":"int", "Platform":"int", "Processor":"int", 
                 "OsVer":"int", "OsPlatformSubRelease":"int", "OsBuildLab":"int", "SkuEdition":"int","Census_MDC2FormFactor":"int", "Census_DeviceFamily":"int", "Census_ChassisTypeName":"int", 
                 "Census_PowerPlatformRoleName":"int","Census_OSVersion":"int", "Census_OSArchitecture":"int", "Census_OSBranch":"int", "Census_OSEdition":"int", "Census_OSSkuName":"int", 
                  "Census_OSInstallTypeName":"int", "Census_OSWUAutoUpdateOptionsName":"int", "Census_GenuineStateName":"int",
                  "Census_ActivationChannel":"int", "Census_FlightRing":"int","Census_PrimaryDiskTypeName":"int"}

da.stringToInt(string_to_int)



<h3>Confirmação da alteração de todas as variáveis String</h3>

In [ ]:
da.getdf().dtypes

<h3>Remoção das colunas que apresentam multicolinearidade</h3>

In [ ]:
da.removeMulticolinearity()

<h3> Total do número de colunas após eliminação </h3>

In [ ]:
len(da.getCols())

<h3> Total do número de registos após a fase de limpeza dos dados </h3>

In [ ]:
da.getRecordsCount()

<h1> Modelos de classificação </h1>

<h3> Criação dos vetores das colunas contínuas, categóricas que não são aceites no teste de Chi Quadrado e categóricas que são aceites </h3>

In [ ]:
cont_cols = ["Census_PrimaryDiskTotalCapacity","Census_TotalPhysicalRAM",\
                "Census_InternalPrimaryDiagonalDisplaySizeInInches","Census_InternalPrimaryDisplayResolutionHorizontal",\
               "Census_InternalBatteryNumberOfCharges","Census_SystemVolumeTotalCapacity"]


categorical_to_drop = ["Census_FirmwareVersionIdentifier","Census_OEMModelIdentifier","CityIdentifier","AVProductStatesIdentifier"]


df = da.getdf().drop(*categorical_to_drop)
cols = df.columns
cat_cols = df.columns
cat_cols.remove("HasDetections")
for col in cont_cols:
    cat_cols.remove(col)


label = "HasDetections"
cols.remove(label)

<h3> Criação e obtenção de resultados para os três modelos idealizados aplicando o método de redução de colunas(p-value < 0.05)</h3> 
                                                                        

In [ ]:
model = MLModel(df,spark,cols,label)
model.setTrainLimit(0.4)
model.reduceCols("p-value",0.05,cat_cols,cont_cols)


model.createPipe("logistic")
accuracy,precision,recall,f1 = model.getPipeMetrics()
print("Logistic results")
print(accuracy,precision,recall,f1)

model.createPipe("tree")
accuracy,precision,recall,f1 = model.getPipeMetrics()
print("Tree results")
print(accuracy,precision,recall,f1)

model.createPipe("forest")
accuracy,precision,recall,f1 = model.getPipeMetrics()
print("Forest results")
print(accuracy,precision,recall,f1)

<h3> Criação e obtenção de resultados para os três modelos idealizados aplicando o método de redução de colunas(numTopFeatures = 10)</h3>

In [ ]:
model = MLModel(df,spark,cols,label)
model.setTrainLimit(0.4)
model.reduceCols("topFeatures",10,cat_cols,cont_cols)


model.createPipe("logistic")
accuracy,precision,recall,f1 = model.getPipeMetrics()
print("Logistic results")
print(accuracy,precision,recall,f1)

model.createPipe("tree")
accuracy,precision,recall,f1 = model.getPipeMetrics()
print("Tree results")
print(accuracy,precision,recall,f1)

model.createPipe("forest")
accuracy,precision,recall,f1 = model.getPipeMetrics()
print("Forest results")
print(accuracy,precision,recall,f1)

<h3> Exemplo de Tunning aplicado ao modelo de regressão logistica aplicando o método de redução de colunas(numTopFeatures = 10)</h3>

In [ ]:
model = MLModel(df,spark,cols,label)
model.setTrainLimit(0.4)
model.reduceCols("topFeatures",10,cat_cols,cont_cols)

maxIterV = [100, 85]
regParamV = [0.0, 0.1]
elasticNetParamV = [0.0, 0.8]
best = model.getLogisticTunningParam(maxIterV,regParamV,elasticNetParamV)

print(t)